In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
dbs = sqlContext.sql("show databases")
dbs.toPandas()

,databaseName
0,default


In [8]:
train = spark.read.csv("train.csv", sep=",", inferSchema=True, header=True)

In [10]:
test = spark.read.csv("test.csv", sep=",", inferSchema=True, header=True)

In [9]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [11]:
type(train)

pyspark.sql.dataframe.DataFrame

In [12]:
train.head(5)

[Row(User_ID=1000001, Product_ID='P00069042', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID='P00248942', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200),
 Row(User_ID=1000001, Product_ID='P00087842', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID='P00085442', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14, Product_Category_3=None, Purchase=1057),
 Row(User_ID=1000002, Product_ID='P0

In [14]:
train.show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

In [18]:
train.count(), test.count()

(550068, 233599)

In [19]:
train.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [20]:
len(train.columns)

12

In [21]:
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [22]:
train.select("User_ID", "Age").show(5)

+-------+----+
|User_ID| Age|
+-------+----+
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000002| 55+|
+-------+----+
only showing top 5 rows



In [25]:
train.select("Product_ID").distinct().count(), test.select("Product_ID").distinct().count()

(3631, 3491)

In [27]:
# Logic in the tutorial gets squishy as to why we are doing this 
diff_cat_in_train_test = test.select('Product_ID').subtract(train.select('Product_ID'))

In [28]:
diff_cat_in_train_test.distinct().count()

46

In [30]:
#Just the other side of it
diff_cat_in_train_test = train.select('Product_ID').subtract(test.select('Product_ID'))
diff_cat_in_train_test.distinct().count()

186

In [31]:
train.crosstab('Age', 'Gender').show()

+----------+-----+------+
|Age_Gender|    F|     M|
+----------+-----+------+
|      0-17| 5083| 10019|
|     46-50|13199| 32502|
|     18-25|24628| 75032|
|     36-45|27170| 82843|
|       55+| 5083| 16421|
|     51-55| 9894| 28607|
|     26-35|50752|168835|
+----------+-----+------+



In [33]:
train.crosstab('Age', 'Occupation').show()

+--------------+-----+-----+-----+----+-----+----+-----+----+----+-----+----+----+-----+-----+----+-----+----+----+-----+---+----+
|Age_Occupation|    0|    1|   10|  11|   12|  13|   14|  15|  16|   17|  18|  19|    2|   20|   3|    4|   5|   6|    7|  8|   9|
+--------------+-----+-----+-----+----+-----+----+-----+----+----+-----+----+----+-----+-----+----+-----+----+----+-----+---+----+
|          0-17| 2134|  387|10951|  18|  237|  15|   93|   0|   0|   35|   0| 807|  144|    0|   0|  113|   0|   0|  139| 29|   0|
|         46-50| 4488| 7089|    0|1584| 2491| 631| 1445| 854|3032| 4662|1124| 261| 2124| 2699|1599|  129|1187|2561| 6664|549| 528|
|         18-25| 9095| 3820| 1649| 717| 4585|   0| 4388| 906|1816| 3944|1085|2500| 4364| 5445|1860|48241|1450|1144| 2078| 14| 559|
|         36-45|13393| 9501|  170|2732| 6848| 427| 5590|2585|7572|10252|1527|1008| 5183| 7508|4126| 1747|3066|4822|18762| 98|3096|
|           55+| 1722| 3139|  134| 143|  322|4870| 1335| 432|1963| 1558| 112| 217| 

In [35]:
train.crosstab('Occupation', 'Gender').show()

+-----------------+-----+-----+
|Occupation_Gender|    F|    M|
+-----------------+-----+-----+
|                0|18112|51526|
|                5| 2220| 9957|
|               10| 4003| 8927|
|               14| 6763|20546|
|               20| 8811|24751|
|                1|17984|29442|
|                6| 8160|12195|
|                9| 5843|  448|
|               13| 1498| 6230|
|                2| 8629|17959|
|               17| 3929|36114|
|               12| 3469|27710|
|                7|10028|49105|
|                3| 7919| 9731|
|               18|  230| 6392|
|               16| 4107|21264|
|               11| 1500|10086|
|                8|  361| 1185|
|               19| 2017| 6444|
|                4|17836|54472|
+-----------------+-----+-----+
only showing top 20 rows



In [37]:
occ_gender = train.crosstab('Occupation', 'Gender')

In [40]:
occ_gender.filter(occ_gender.F > occ_gender.M).show()

+-----------------+----+---+
|Occupation_Gender|   F|  M|
+-----------------+----+---+
|                9|5843|448|
+-----------------+----+---+



In [41]:
train.select('Age', 'Gender').dropDuplicates().show()

+-----+------+
|  Age|Gender|
+-----+------+
|51-55|     F|
|18-25|     M|
| 0-17|     F|
|46-50|     M|
|18-25|     F|
|  55+|     M|
|  55+|     F|
|36-45|     M|
|26-35|     F|
| 0-17|     M|
|36-45|     F|
|51-55|     M|
|26-35|     M|
|46-50|     F|
+-----+------+



In [42]:
train.dropna().count()

166821

In [43]:
train.count()

550068

In [44]:
# Fill in values
train.fillna(-1).show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

In [46]:
train.filter(train.Purchase > 15000).count()

110523

In [55]:
# Note you can do multiple columns but not multiple summaries on the same column apparently.  Last one overwrites the first
train.groupby('Age').agg({'Purchase': 'mean', 'Gender': 'max'}).show()

+-----+-----------------+-----------+
|  Age|    avg(Purchase)|max(Gender)|
+-----+-----------------+-----------+
|18-25|9169.663606261289|          M|
|26-35|9252.690632869888|          M|
| 0-17|8933.464640444974|          M|
|46-50|9208.625697468327|          M|
|51-55|9534.808030960236|          M|
|36-45|9331.350694917874|          M|
|  55+|9336.280459449405|          M|
+-----+-----------------+-----------+



In [56]:
t1 = train.sample(False, 0.2, 42)
t2 = train.sample(False, 0.2, 43)

In [57]:
t1.count(), t2.count()

(109657, 109532)

In [60]:
# Important note: Spark 1 could use .map on a dataframe now it is only available on the underlying rdd structure
train.select('User_ID').rdd.map(lambda x:(x,1)).take(5)

[(Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000002), 1)]

In [63]:
train.orderBy(train.Purchase.desc(),train.User_ID.asc()).show(5)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1001474| P00052842|     M|26-35|         4|            A|                         2|             1|                10|                15|              null|   23961|
|1002272| P00052842|     M|26-35|         0|            C|                         1|             0|                10|                15|              null|   23961|
|1003160| P00052842|     M|26-35|        17|            C|                         3|             0|                10|                15|              null|   23961

In [65]:
train.withColumn('Purchase_new', train.Purchase/2.0).select('Purchase', 'Purchase_new').toPandas()

,Purchase,Purchase_new
0,8370,4185.0
1,15200,7600.0
2,1422,711.0
3,1057,528.5
4,7969,3984.5
...,...,...
550063,368,184.0
550064,371,185.5
550065,137,68.5
550066,365,182.5


In [66]:
test.drop('Comb').columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3']

In [67]:
#UDF EXAMPLE
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()

46

In [77]:
not_found_cat = diff_cat_in_train_test.distinct().rdd.map(lambda x: x[0]).collect()

In [79]:
len(not_found_cat)

46

In [80]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
F1 = udf(lambda x: '-1' if x in not_found_cat else x, StringType())

In [84]:
k = test.withColumn("NEW_Product_ID", F1(test["Product_ID"])).select("NEW_Product_ID")

In [89]:
diff_cat_in_train_test = k.select('NEW_Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()

1

In [90]:
diff_cat_in_train_test.distinct().collect()

[Row(NEW_Product_ID='-1')]

### SQL Queries on a DataFrame

In [96]:
# Example from the tutorial is wrong: train.registerAsTable('train_table')  Throws an error that the registerastable doesn't exist
train.registerTempTable("train_table")

In [97]:
sqlContext.sql('select Product_ID from train_table').show(5)

+----------+
|Product_ID|
+----------+
| P00069042|
| P00248942|
| P00087842|
| P00085442|
| P00285442|
+----------+
only showing top 5 rows



In [98]:
sqlContext.sql('select Age, max(Purchase) from train_table group by Age').show()

+-----+-------------+
|  Age|max(Purchase)|
+-----+-------------+
|18-25|        23958|
|26-35|        23961|
| 0-17|        23955|
|46-50|        23960|
|51-55|        23960|
|36-45|        23960|
|  55+|        23960|
+-----+-------------+

